In [1]:
import time
import re
import json
import statistics as stats
import requests as req
import numpy as np
import pandas as pd
from unidecode import unidecode
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains as AC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
import tqdm
import feedparser
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [2]:
# Configurar opciones de Chrome
chrome_options = Options()

chrome_options.add_argument(r"user-data-dir=C:\\Users\\User\\AppData\\Local\\Google\\Chrome\\User Data")
chrome_options.add_argument("--remote-allow-origins=*")
chrome_options.add_experimental_option("useAutomationExtension", False) 
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_argument("user-data-dir=selenium-cookies")
chrome_options.add_argument('--headless')  # Habilitar si no queremos ver la ventana
chrome_options.add_experimental_option("detach", True)  # Esta opción corrige el error de cierre repentino
chrome_options.add_argument('--start-minimized')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--disable-infobars')

In [3]:
driver = webdriver.Chrome(options=chrome_options)

In [4]:
def scrapeo_flora_data(driver, url):
    driver.get(url)

    # Obtener el tipo de flora de la etiqueta en la parte superior de la página
    tipo_flora = driver.find_element(By.XPATH, '//*[@id="panListadoFiltros"]/ul/li').text.strip()

    figures = driver.find_elements(By.XPATH, '//*[@id="panResultados"]/div/figure')

    resultados = []

    for figure in figures:
        titulo = figure.find_element(By.XPATH, './/figcaption[1]/div/p[1]').text.strip()
        fecha = figure.find_element(By.XPATH, './/figcaption[1]/div/p[2]').text.strip()
        autor = figure.find_element(By.XPATH, './/figcaption[1]/div/p[3]').text.strip()

        try:
            href = figure.find_element(By.XPATH, './/figcaption[1]/div/p[3]/a').get_attribute('href')
        except NoSuchElementException:
            # Manejo de excepción si no se encuentra el elemento href
            href = None

        resultados.append([titulo, fecha, autor, tipo_flora, href])

    return resultados





In [5]:
# Ejemplo de uso
urls = [
#PRIMERA PAGINA
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_547',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_365',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_389',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_424',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_394',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_390',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_359',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_4',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_302',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_343',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_388',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_347',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_875',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_341',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_296',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_436',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_540',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_475',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_426',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_306',

#SEGUNDA PAGINA
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_552',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_370',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_277',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_454',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_360',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_548',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_382',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_414',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_846',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_157',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_566',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_346',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_441',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_284',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_315',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_458',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_848',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_429',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_332',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_384',

#TERCERA PAGINA
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_448',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_863',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_379',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_381',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_289',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_490',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_583',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_262',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_420',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_539',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_457',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_282',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_293',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_319',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_294',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_349',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_421',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_567',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_358',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_435',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_402',

#CUARTA PAGINA
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_845',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_396',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_278',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_843',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_439',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_586',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_446',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_874',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_352',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_428',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_860',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_295',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_861',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_476',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_580',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_380',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_473',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_320',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_356',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_550',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_498',

#QUINTA PAGINA
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_432',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_321',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_427',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_438',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_281',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_361',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_395',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_494',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_299',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_369',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_461',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_489',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_403',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_546',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_397',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_563',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_376',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_280',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_378',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_334',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_323',

#SEXTA PAGINA
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_335',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_308',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_351',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_472',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_551',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_312',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_478',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_588',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_336',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_301',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_492',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_286',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_276',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_560',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_368',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_353',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_545',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_377',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_300',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_265',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_363',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_303',

#SEPTIMA PAGINA
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_318',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_288',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_345',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_287',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_392',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_442',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_371',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_405',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_305',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_437',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_285',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_450',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_309',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_274',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_544',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_431',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_578',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_311',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_595',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_538',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_444',

#OCTAVA PAGINA
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_862',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_344',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_307',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_338',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_348',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_362',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_263',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_585',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_385',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_314',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_462',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_430',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_541',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_491',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_549',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_328',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_327',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_387',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_477',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_400',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_419',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_367',

#NOVENA PAGINA
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_849',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_313',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_366',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_275',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_283',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_542',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_398',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_331',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_364',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_497',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_568',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_354',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_298',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_5',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_451',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_333',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_386',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_404',
'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_562',

]

In [7]:
# Lista para almacenar los resultados de todas las flores
resultados_globales = []

# Ejecutar la función para cada URL de flor
for url_flor in urls:
    resultados_flor = scrapeo_flora_data(driver, url_flor)
    time.sleep(4)
    resultados_globales.extend(resultados_flor)

# Convertir la lista de resultados globales en un DataFrame
columns = ['Titulo', 'Tecnica_Fecha', 'Autor', 'Flora', 'Href']
flora_df_global = pd.DataFrame(resultados_globales, columns=columns)

# Imprimir el DataFrame
flora_df_global

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome-headless-shell=120.0.6099.199)
Stacktrace:
0   chromedriver                        0x00000001010244dc chromedriver + 4162780
1   chromedriver                        0x000000010101c664 chromedriver + 4130404
2   chromedriver                        0x0000000100c73bc0 chromedriver + 293824
3   chromedriver                        0x0000000100c5b0b0 chromedriver + 192688
4   chromedriver                        0x0000000100c5afd8 chromedriver + 192472
5   chromedriver                        0x0000000100cf92b0 chromedriver + 840368
6   chromedriver                        0x0000000100cad6bc chromedriver + 530108
7   chromedriver                        0x0000000100cae930 chromedriver + 534832
8   chromedriver                        0x0000000100fe9e08 chromedriver + 3923464
9   chromedriver                        0x00000001

In [10]:
# Lista para almacenar los resultados de todas las flores
resultados_globales = []

# Ejecutar la función para cada URL de flor
for url_flor in urls:
    try:
        resultados_flor = scrapeo_flora_data(driver, url_flor)
        time.sleep(8)
        resultados_globales.extend(resultados_flor)
    except (TimeoutException, StaleElementReferenceException) as e:
        print(f"Error en la URL {url_flor}: {str(e)}")

In [11]:
# Convertir la lista de resultados globales en un DataFrame
columns = ['Titulo', 'Tecnica_Fecha', 'Autor', 'Flora', 'Href']
flora_df_global = pd.DataFrame(resultados_globales, columns=columns)

# Imprimir el DataFrame
flora_df_global

,Titulo,Tecnica_Fecha,Autor,Flora,Href
